In [1]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
env=AngryBird()

# 240 coeffs

In [2]:
seconds = np.arange(11)
pos = np.zeros((11, 2))
pos[:, 0] = seconds * 10
pos[:, 1] = seconds * 20 - 2 * seconds**2

In [3]:
mean_states = np.load("mean_states.npy")

In [4]:
cor = np.array([[0, 0, 0],
               [0, 3, 2],
               [0, 1, 0]])

In [5]:
coeffs = np.load("cc.npy")
coeffs2 = np.load("cc2.npy")

In [6]:
def main(state, coeffs, coeffs2):
    t = int(state[0])
    a, b =np.sign(state[1:] - mean_states[t]).astype(int)
    i = cor[a][b]
    if t < 9:
        return coeffs[t, i, :2]*pos[t]-coeffs[t, i, 2:4]*state[1:] + coeffs[t, i, 4:]
    else:
        return coeffs[t, i, :2]*pos[t]-coeffs[t, i, 2:4]*state[1:] + coeffs[t, i, 4:] + coeffs2*state[1:]**2 * (i < 0.5)

In [7]:
from time import time
nul=np.zeros(2)
m = 100
n = 1000
vents = np.load("vents.npy")
ecart = 0.0001
cos_min = 400.
while cos_min > 100:
    t = -time()
    for i in range(10):
        for l in range(4):
            for k in range(6):
                coeff_change = np.linspace(coeffs[i][l][k]-ecart, coeffs[i][l][k]+ecart, 9)
                costs = np.zeros(9)
                for j in range(9):
                    coeffs[i][l][k] = coeff_change[j]
                    for z in range(m):
                        state = env.reset()
                        env.vent = vents[z]
                        for _ in range(10):
                            force = main(state, coeffs, coeffs2)
                            state, _, _, _ = env.step(force)
                        env.step(nul)
                        costs[j] += env.cost.sum()
                coeffs[i][l][k] = coeff_change[np.argmin(costs)]
    for k in range(2):
        coeff_change = np.linspace(coeffs2[k]-ecart, coeffs2[k]+ecart, 9)
        costs = np.zeros(9)
        for j in range(9):
            coeffs2[k] = coeff_change[j]
            for z in range(m):
                state = env.reset()
                env.vent = vents[z]
                for _ in range(10):
                    force = main(state, coeffs, coeffs2)
                    state, _, _, _ = env.step(force)
                env.step(nul)
                costs[j] += env.cost.sum()
        coeffs2[k] = coeff_change[np.argmin(costs)]

    cos = 0.
    for z in range(n):
        state = env.reset(False)
        env.vent = vents[z]
        for _ in range(10):
            force = main(state, coeffs, coeffs2)
            state, _, _, _ = env.step(force)
        env.step(nul)
        cos += env.cost.sum()
    cos /= n
    t += time()
    print(m, n, ecart, costs.min()/m, cos, t/60)
    if cos < cos_min:
        cos_min = cos
        np.save("ccc", coeffs)
        np.save("ccc2", coeffs2)
    else:
        cos_min = cos + 20
        m += 100
        n += 100
        ecart /= 2

100 1000 0.0001 109.96406380455231 116.72615629645483 8.858468055725098
100 1000 0.0001 109.90927579258636 116.75708521627688 8.865796999136608
200 1100 5e-05 116.37402174424369 117.78004892695179 17.748656237125395
200 1100 5e-05 116.35903473149007 117.78060930224966 17.936009701093038
300 1200 2.5e-05 113.39953230066952 116.31563374982831 26.489291604359945
300 1200 2.5e-05 113.39411216621471 116.31480960208833 26.525990255673726
300 1200 2.5e-05 113.3900504352464 116.30868299524569 26.5227032661438
300 1200 2.5e-05 113.38733276823747 116.30579248274454 26.525521687666576
300 1200 2.5e-05 113.38475580488344 116.30224463453271 26.528412481149036
300 1200 2.5e-05 113.38092426840548 116.30441528186785 26.53862011830012
400 1300 1.25e-05 121.05759872328294 115.30790725296191 35.35630201101303
400 1300 1.25e-05 121.0559816514312 115.30503547665157 35.367327551047005
400 1300 1.25e-05 121.05442983891834 115.30275616196386 35.37208218971888
400 1300 1.25e-05 121.05295483701462 115.300267261

KeyboardInterrupt: 

In [44]:
n = 10000
cos = 0.
X = []
Y = []
for z in range(n):
    state = env.reset(False)
    env.vent = vents[z]
    for y in range(9):
        force = main(state, coeffs, coeffs2)
        state, _, _, _ = env.step(force)
    force = main(state, coeffs, coeffs2)
    X.append(np.copy(state[1:]))
    state, _, _, _ = env.step(force)
    Y.append(-state[1:])
X = np.array(X)
Y = np.array(Y)